In [1]:
import cv2
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

In [ ]:
def plt_images(orig_image, orig_title, processed_image, processed_title, cmap='gray'):
    # Visualize undirstorsion
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(25, 10))
    ax1.set_title(orig_title, fontsize=30)
    ax1.imshow(orig_image)
    ax2.set_title(processed_title, fontsize=30)
    ax2.imshow(processed_image, cmap='gray')
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
img = mpimg.imread("./images/test3.jpg") 
im = cv2.imread("./images/test3.jpg") 
plt.imshow(im)
img.shape

In [ ]:
src = np.float32([(550, 460),     # top-left
                  (150, 720),     # bottom-left
                  (1200, 720),    # bottom-right
                  (770, 460)])  
dst = np.float32([(100, 0),
                 (100, 720),
                 (1100, 720),
                 (1100, 0)])
M = cv2.getPerspectiveTransform(src, dst)
M_inv = cv2.getPerspectiveTransform(dst,src)

img_size=(1280, 720)
flags=cv2.INTER_LINEAR
mm=cv2.warpPerspective(img,M, img_size, flags=flags)
#minv=cv2.warpPerspective(img, M_inv, img_size, flags=flags)
plt_images(img, 'Source image', mm, 'mm image')
#plt_images(img, 'Source image', minv, 'minv image')

In [ ]:
hls = cv2.cvtColor(mm, cv2.COLOR_RGB2HLS)
hsv = cv2.cvtColor(mm, cv2.COLOR_RGB2HSV)
h_channel = hls[:,:,0]
l_channel = hls[:,:,1]
s_channel = hls[:,:,2]
v_channel = hsv[:,:,2]
print(v_channel)
plt_images(mm, 'Source image', hls, 'HLS image')
plt_images(mm, 'Source image', hsv, 'HSV image')

**Thresholding function definition**


In [ ]:
def threshold_rel(img, lo, hi):
  #getting the maximum and minimum pixels intesties
    vmin = np.min(img)
    print(vmin)
    vmax = np.max(img)
    print(vmax)
    
    vlo = vmin + (vmax - vmin) * lo
    vhi = vmin + (vmax - vmin) * hi
    return np.uint8((img >= vlo) & (img <= vhi)) * 255


In [ ]:
right_lane = threshold_rel(l_channel, 0.8, 1.0)
plt_images(hls, 'Source image', right_lane, 'thresholded image')

In [ ]:
def threshold_abs(img, lo, hi):
    return np.uint8((img >= lo) & (img <= hi)) * 255

In [ ]:
right_lane = threshold_rel(l_channel, 0.8, 1.0)
print(l_channel)
right_lane[:,:750] = 0
plt_images(hls, 'Source image', right_lane, 'thresholded image')


In [ ]:
left_lane = threshold_abs(h_channel, 20, 30)
left_lane &= threshold_rel(v_channel, 0.7, 1.0)
left_lane[:,550:] = 0
plt_images(hls, 'Source image', left_lane, 'thresholded image')


In [ ]:
mm = left_lane | right_lane
plt_images(hls, 'Source image', mm, 'thresholded image')